In [1]:
import pygame
import os

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
SCREEN_HEIGHT=600
SCREEN_WIDTH=1100
SCREEN=pygame.display.set_mode((SCREEN_WIDTH,SCREEN_HEIGHT))

In [3]:
running=[pygame.image.load(os.path.join("DinoRun1.png")),pygame.image.load(os.path.join("DinoRun2.png"))]
jumping=pygame.image.load(os.path.join("DinoJump.png"))
ducking=[pygame.image.load(os.path.join("DinoDuck1.png")),pygame.image.load(os.path.join("DinoDuck2.png"))]
dying=pygame.image.load(os.path.join("DinoDead.png"))
small_cactus=[pygame.image.load(os.path.join("SmallCactus1.png")),pygame.image.load(os.path.join("SmallCactus2.png")),pygame.image.load(os.path.join("SmallCactus3.png"))]
large_cactus=[pygame.image.load(os.path.join("LargeCactus1.png")),pygame.image.load(os.path.join("LargeCactus2.png")),pygame.image.load(os.path.join("LargeCactus3.png"))]
cacti=small_cactus+large_cactus
birds=[pygame.image.load(os.path.join("Bird1.png")),pygame.image.load(os.path.join("Bird2.png"))]
cloud=pygame.image.load(os.path.join("Cloud.png"))
track=pygame.image.load(os.path.join("Track.png"))
reset=pygame.image.load(os.path.join("Reset.png"))
game_over=pygame.image.load(os.path.join("GameOver.png"))

In [4]:
end_game=False

In [5]:
class Dinosaur:
    X_pos= 120
    Y_pos= 310
    score=0
    def __init__(self):
        self.duck_img=ducking
        self.jump_img=jumping
        self.run_img=running
        self.jump_vel=8.5
        self.am_ducking=False;
        self.am_jumping=False;
        self.am_running=True;

        self.steps=0
        self.timer=0
        self.image=self.jump_img
        self.dino_rect=self.image.get_rect()
        self.dino_rect.x=self.X_pos;
        self.dino_rect.y=self.Y_pos;

    def update(self,user_input):
        self.timer+=1
        if self.am_ducking:
            self.duck()
        if self.am_running:
            self.run()
        if self.am_jumping:
            self.jump()
        if self.steps>=10:
            self.steps=0
        if self.timer%5==0: 
            self.score+=1
        if user_input[pygame.K_UP] and not self.am_jumping:
            self.am_ducking=False
            self.am_jumping=True
            self.am_running=False
        elif user_input[pygame.K_DOWN] and not self.am_jumping:
            self.am_ducking=True
            self.am_jumping=False
            self.am_running=False
        elif user_input[pygame.K_DOWN] or self.am_ducking or not self.am_jumping:
            self.am_ducking=False
            self.am_jumping=False
            self.am_running=True
        
    def duck(self):
        self.image=self.duck_img[self.steps//5]
        self.dino_rect=self.image.get_rect()
        self.dino_rect.x=self.X_pos
        self.dino_rect.y=self.Y_pos+30
        self.steps+=1

    def run(self):
        self.image=self.run_img[self.steps//5]
        self.dino_rect=self.image.get_rect()
        self.dino_rect.x=self.X_pos
        self.dino_rect.y=self.Y_pos
        self.steps+=1

    def jump(self):
        self.image=self.jump_img
        # self.dino_rect=self.image.get_rect()
        if self.am_jumping:
            self.dino_rect.y-=self.jump_vel*4
            self.jump_vel-=0.8
        if self.dino_rect.y >= self.Y_pos:
            self.dino_rect.y = self.Y_pos
            self.jump_vel = 8.5
            self.am_jumping = False


    def draw(self,SCREEN):
        SCREEN.blit(self.image,(self.dino_rect.x,self.dino_rect.y))
        font = pygame.font.SysFont('Verdana', 10, bold=True)
        text_surface = font.render("SCORE : "+str(self.score), True, (0, 0, 0)) 
        text_rect = text_surface.get_rect(center=(1000, 40))
        SCREEN.blit(text_surface,text_rect)

In [6]:
import random
import math

class Obstacle:
    def __init__(self):
        self.steps = 0

    def distance(self, x1, y1, x2, y2) -> bool:
        return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) <= 40

    def collide(self, dino_rect):
        global end_game
        for rect in self.collision_rects():
            if self.distance(rect.centerx, rect.centery, dino_rect.centerx, dino_rect.centery):
                end_game = True

    def collision_rects(self):
        # overrridden
        return []


class Cactus(Obstacle):
    def __init__(self):
        super().__init__()
        self.image_1 = random.choice(cacti)
        self.image_2 = random.choice(cacti)

        self.image_rect_1 = self.image_1.get_rect()
        self.image_rect_2 = self.image_2.get_rect()

        self.image_rect_1.x = 600
        self.image_rect_2.x = 1100

        self.image_rect_1.y = 400 - self.image_1.get_height()
        self.image_rect_2.y = 400 - self.image_2.get_height()

    def draw(self, SCREEN):
        if self.image_rect_1.x <= -500:
            self.image_1 = random.choice(cacti)
            self.image_rect_1.x = 1100 + self.image_1.get_width()
            self.image_rect_1.y = 400 - self.image_1.get_height()
        if self.image_rect_2.x <= -500:
            self.image_2 = random.choice(cacti)
            self.image_rect_2.x = 1100 + self.image_2.get_width()
            self.image_rect_2.y = 400 - self.image_2.get_height()
        else:
            self.image_rect_1.x -= 14
            self.image_rect_2.x -= 14

        SCREEN.blit(self.image_1, (self.image_rect_1.x, self.image_rect_1.y))
        SCREEN.blit(self.image_2, (self.image_rect_2.x, self.image_rect_2.y))

    def collision_rects(self):
        return [self.image_rect_1, self.image_rect_2]


class Bird(Obstacle):
    def __init__(self):
        super().__init__()
        self.image_3 = random.choice(birds)
        self.image_4 = cloud

        self.image_rect_3 = self.image_3.get_rect()
        self.image_rect_4 = self.image_4.get_rect()

        self.image_rect_3.x = 1600
        self.image_rect_3.y = 350 - self.image_3.get_height()

        self.image_rect_4.x = 800
        self.image_rect_4.y = 80

    def draw(self, SCREEN):
        self.steps += 1
        if self.steps >= 10:
            self.steps = 0
        self.image_3 = birds[self.steps // 5]

        if self.image_rect_3.x <= -500:
            self.image_3 = random.choice(birds)
            self.image_rect_3.x = 1100 + self.image_3.get_width()
            self.image_rect_3.y = 350 - self.image_3.get_height()
        if self.image_rect_4.x <= -100:
            self.image_rect_4.x = 1100 + self.image_3.get_width()
        else:
            self.image_rect_3.x -= 20
            self.image_rect_4.x -= 15

        SCREEN.blit(self.image_3, (self.image_rect_3.x, self.image_rect_3.y))
        SCREEN.blit(self.image_4, (self.image_rect_4.x, self.image_rect_4.y))

    def collision_rects(self):
        return [self.image_rect_3]


In [7]:
class Track:
    track_X_pos=0
    track_Y_pos=390

    def __init__(self):
        self.track_img_1=track
        self.track_rect_1=self.track_img_1.get_rect()
        self.track_rect_1.x=self.track_X_pos
        self.track_rect_1.y=self.track_Y_pos
        self.track_img_2=track
        self.track_rect_2=self.track_img_2.get_rect()
        self.track_rect_2.x=self.track_X_pos+2404
        self.track_rect_2.y=self.track_Y_pos

    def draw(self,SCREEN):
        if self.track_rect_1.x <-2380:
            self.track_rect_1.x=2404
        if self.track_rect_2.x <-2380:
            self.track_rect_2.x=2404
        else:
            self.track_rect_1.x-=14
            self.track_rect_2.x-=14
        SCREEN.blit(self.track_img_1,(self.track_rect_1.x,self.track_rect_1.y))
        SCREEN.blit(self.track_img_2,(self.track_rect_2.x,self.track_rect_2.y))


In [8]:
class Button():
	def __init__(self, x, y, image, scale):
		width = image.get_width()
		height = image.get_height()
		self.image = pygame.transform.scale(image, (int(width * scale), int(height * scale)))
		self.rect = self.image.get_rect()
		self.rect.topleft = (x, y)
		self.clicked = False

	def draw(self, surface):
		action = False
		#get mouse position
		pos = pygame.mouse.get_pos()

		#check mouseover and clicked conditions
		if self.rect.collidepoint(pos):
			if pygame.mouse.get_pressed()[0] == 1 and self.clicked == False:
				self.clicked = True
				action = True

		if pygame.mouse.get_pressed()[0] == 0:
			self.clicked = False

		#draw button on screen
		surface.blit(self.image, (self.rect.x, self.rect.y))
		return action

In [9]:
def display_menu(screen,k):
    global end_game
    font = pygame.font.SysFont('Verdana', 60, bold=True)
    font1 = pygame.font.SysFont('Verdana', 40, bold=True)
    text_surface = font.render("GAME OVER!", True, (255, 70, 0)) 
    text_rect = text_surface.get_rect(center=(550, 200))
    screen.blit(text_surface, text_rect)
    text_surface_1=font1.render("SCORE : "+str(k), True, (0, 0, 0)) 
    text_rect_1 = text_surface_1.get_rect(center=(550, 300))
    screen.blit(text_surface_1, text_rect_1)
    buttonImg=pygame.image.load('reset.png')
    button=Button(550,400,buttonImg,0.8)
    k=button.draw(screen)
    if k:
        end_game=False;

In [10]:
def main():
    run=True;
    clock=pygame.time.Clock()
    player=Dinosaur()
    track=Track()
    cactus=Cactus()
    bird=Bird()
    k=0
    while run:
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run=False

        SCREEN.fill((255,255,255))

        if end_game:
            display_menu(SCREEN,k)
            if not end_game:
                player = Dinosaur()
                track = Track()
                cactus = Cactus()
                bird = Bird()
        else:
            user_input=pygame.key.get_pressed()
            player.draw(SCREEN)
            track.draw(SCREEN)
            cactus.draw(SCREEN)
            bird.draw(SCREEN)
            player.update(user_input)
            cactus.collide(player.dino_rect)
            bird.collide(player.dino_rect)
            k=player.score
        clock.tick(30)
        pygame.display.update()

main()
pygame.quit()